Here we are performing preprocessing with the MUSE data. 
We need to take care of the "nan" (Dead pixel) and rotate the image cube.

In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
from deconvbench import Deconvbench
from mpl_toolkits.axes_grid1 import make_axes_locatable
from amiral import instructment, utils, parameter, gradient, minimisation, array, plotting
from astropy.visualization import make_lupton_rgb
import scipy
import astropy.io as astropy


# from plotting import plot_PSF_PSD as amiral_plt
from scipy.optimize import minimize 

%matplotlib inline

import tools

rcParams["figure.figsize"] = 20,33

In [ ]:
# Function for getting snr
def snr_map (array, n_sky, n_ron): 
    
    # S/N = S/N_tot = S /  sqrt (S+Sky+Dark +N_ron^2)
    # As the RON is small, we can ignore that 
    # Sky: background -> get the mean value of the background from the corner
    
    dimension = np.shape(array)[0]
    snr_map =  np.zeros((dimension,dimension))
    snr_map[:,:] = array[:,:] / np.sqrt(array[:,:] + n_sky + n_ron**2)
    
    return snr_map

def get_snr (array):
    
    mean = np.mean(array)
    sig2 = np.std(array)
    
    snr = mean / sig2
    
    return snr

def resize_array (array, size, cent = None):
    """
    
    Resize the array to a given size. Cent is optional.
    
    If cent is an input, it contains the centre of the image
    
    """
    if array.ndim == 3: 
        if cent == None:
            cent = [np.shape(array[0])[0]//2, np.shape(array[0])[1]//2]
            
        _len = np.shape(array)[0]
        resize_cube = np.zeros((_len,size, size))
        
        for i in range (_len):
            resize_cube[i] = array[i][cent[0]- size//2:cent[0] +size//2,\
                                      cent[0] - size//2:cent[0] +size//2]
        return resize_cube
    
    else:
        if cent == None:
            cent = [np.shape(array)[0]//2, np.shape(array)[1]//2]
            
            resize_array = array[cent[1] - size//2:cent[1] +size//2, \
                             cent[0] - size//2:cent[0] +size//2]

        return resize_array

def drop_frame (cube, value):
    """
    Get the indice for array which contains too many "nan" values
    """
    dropped_list = []
    
    for i in range (len(cube)):
    
        nan_ind = np.argwhere(np.isnan(cube[i]))
        if len(nan_ind) > value:
            dropped_list.append(i)
        
    return dropped_list 


In [ ]:
# def _inteploate_check_nan (array, ind, radius):
#     """
#     internal function to check the number of nan value in a given range. 
    
#     If it is more than 1, reduce the radius by 1. 
    
#     """
#     _y, _x = ind 
#     print(ind)
    
#     if len(ind) > 1: 
#         print("There is more than one nan value in a given range")

#         radius -= 1
        
#         _neighbour = np.append(array[_y-radius:_y+radius],array[_x-radius:_x+radius])
#         _neighbour_ind = np.argwhere(np.isnan(_neighbour))
        
#         return _neighbour,_neighbour_ind
        
#     else: 
        
#         return array, ind

# def inteploate_nan (array, radius):
    
#     nan_ind = np.argwhere(np.isnan(array))
        
#     for i in range (len(nan_ind)):
#         print(i)
        
#         # Retrieve the coordinate of the nan pixel
#         _y, _x = nan_ind[i]
        
#         _neighbour = np.append(array[_y-radius:_y+radius],array[_x-radius:_x+radius])
        
#         _ind = np.argwhere(np.isnan(_neighbour))
#         print(_ind)
        
#         _neighbour, _neighbour_ind = _inteploate_check_nan (array, nan_ind[i], radius)
        
#         # Remove the nan pixel value that we would like to replace
#         _neighbour = np.delete(_neighbour, _neighbour_ind)
        
#         array[_y, _x] = np.mean(_neighbour)
#         print("Value of the new pixel: %f " %(array[_y, _x]))
        
        
#     return array

In [ ]:
# PATH
wdir = "/Users/alau/Data/MUSE_DATA/HD_146233/"
data_cube = ["HD_146233_DATACUBE_FINAL_2019-07-08T02_48_09.506"]

In [ ]:
# Variable
muse_pix_scale = 0.025 # arcsecond per pixel

Have a look at the actual data before performing any data analysis

In [ ]:
name = data_cube[0]
_cube = fits.open(wdir+name+".fits")
_cube.info()

cube = _cube[1].data
# cube_1 = _cube[1].data
cube_resize = resize_array(cube,250,cent=(139,132))


In [ ]:
plt.imshow(cube_resize[2])

In [ ]:
array_resize = resize_array(cube[1],200)

In [ ]:
print("Dimension of the original image: %s" %(np.shape(cube[0]),))
print("Dimension of the cropped image: %s" %(np.shape(array_resize),))

fig, ax = plt.subplots(1,2)
xlimt = 200//2 * muse_pix_scale
ylimt = 200//2 * muse_pix_scale

ax[1].imshow(array_resize, interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
ax[1].set_title('Cropped Image', fontsize = '16')
ax[1].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

xlimt = np.shape(cube[0])[0]//2 * muse_pix_scale
ylimt = np.shape(cube[0])[1]//2 * muse_pix_scale

# ax[0].imshow(cube[0],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
ax[0].imshow(cube[1],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
ax[0].set_title('Original Image', fontsize = '16')
ax[0].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

In [ ]:
wdir = "/Users/alau/Data/MUSE_DATA/Ganymede/2019sep08/"

data_cube = ["Ganymede_DATACUBE_FINAL_2019-07-08T02_06_47.922_cube_1",\
        "Ganymede_DATACUBE_FINAL_2019-07-08T02_06_47.922_cube_2"]

name = data_cube[0]

_cube = fits.open(wdir+name+".fits")
_cube.info()

cube = _cube[1].data
_cube.pop(2)
_cube.info()

Perform cropping

In [ ]:
cube_resize = resize_array(cube,250,cent=(139,132))

plt.imshow(cube_resize[0])

In [ ]:
data_cube = ["Ganymede_20190708_cube_1_cropped_cube","Ganymede_20190708_cube_2_cropped_cube"]

name = data_cube[0]

_cube[1].data = cube_resize
# array.save_fits(_cube,name,wdir)

Now we know we have 2 cubes which have been cropped. Now we need to drop some frames if they are corrupted --> then we can perform binning. 

In [ ]:
_ls = drop_frame(cube_resize,35)
print(_ls)

nan_ind = np.argwhere(np.isnan(cube_resize[-1]))
print(len(nan_ind))

In [ ]:
plt.imshow(cube_resize[0])

In [ ]:
_new_cube = np.zeros((cube_resize.shape[0]-len(_ls),cube_resize.shape[1],cube_resize.shape[2]))

_new_cube.shape

In [ ]:
_new_cube = cube_resize[_ls[0]+1:_ls[1]]
print(_new_cube.shape)

plt.imshow(_new_cube[0])

In [ ]:
data_cube = ["Ganymede_20190708_cube_1_cropped_cube_clean","Ganymede_20190708_cube_2_cropped_cube_clean"]
name = data_cube[0]

wvl_min = _cube[1].header['CRVAL3']*1e-10/1e-9
wvl_slice = _cube[1].header['CD3_3']*1e-10/1e-9

print("Starting wavelegnth: %.3f[nm]" %(wvl_min))
print("Wavelength slice: %.3f[nm]" %(wvl_slice))
print("New starting wavelength: %.3f[A]" %((wvl_min+wvl_slice)*1e1))

# array_resize = resize_array(cube[1],250,cent=(139,132))
_cube[1].data = _new_cube

_cube[1].header['CRVAL3'] = (wvl_min+wvl_slice)*1e1

# array.save_fits(_cube,name, wdir)

In [ ]:
array_resize.shape

In [ ]:
fig, ax = plt.subplots(1,2)
xlimt = 200//2 * muse_pix_scale
ylimt = 200//2 * muse_pix_scale

ax[1].imshow(cube[1], interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
ax[1].set_title('Original Image', fontsize = '16')
ax[1].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

xlimt = np.shape(cube[0])[0]//2 * muse_pix_scale
ylimt = np.shape(cube[0])[1]//2 * muse_pix_scale

# ax[0].imshow(cube[0],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
ax[0].imshow(array_resize,interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
ax[0].set_title('Cropped Image', fontsize = '16')
ax[0].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

In [ ]:
plt.imshow(array_resize)

In [ ]:
# nan_ind = np.argwhere(np.isnan(array_resize))
# print("Nan index: %s" %(nan_ind,))


# x, y = nan_ind[0]

# print(x,y)

# pix_list = np.append(array_resize[x-9:x+9],array_resize[y-9:y+9])
# _test = np.argwhere(np.isnan(pix_list))

# _pix_list = np.delete(pix_list, _test)

# _ind = np.argwhere(np.isnan(_pix_list))

# print(len(_ind))


# radius = 10 

# radius -=1

# print(radius)

In [ ]:
# array.save_fits(_cube,name,wdir)

In [ ]:
# wdir = "/Users/alau/Data/MUSE_DATA/Ganymede/2019sep08/"
# binned_cube = ["Ganymede_20190708_cube_1_binned_10", "Ganymede_20190708_cube_2_binned_10"]

In [ ]:
# name = binned_cube[1]

# _cube = fits.open(wdir+name+".fits")
# _cube.info()

# _binned_cube = _cube[1].data
# _cube.info()


# plt.imshow(_binned_cube[1])

In [ ]:
# _test_cube_1 = np.zeros((np.shape(_binned_cube)))

# print(np.shape(_binned_cube))

# for i in range (82): 
#     _test_cube_1[i] = inteploate_nan(_binned_cube[i], radius = 4)

In [ ]:
# fig, ax = plt.subplots(1,3)
# xlimt = 200//2 * muse_pix_scale
# ylimt = 200//2 * muse_pix_scale

# ax[1].imshow(_binned_cube[0], interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[1].set_title('Original Image', fontsize = '16')
# ax[1].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

# xlimt = np.shape(cube[0])[0]//2 * muse_pix_scale
# ylimt = np.shape(cube[0])[1]//2 * muse_pix_scale

# # ax[0].imshow(cube[0],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[0].imshow(_test_cube_1[0],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[0].set_title('Filled image', fontsize = '16')
# ax[0].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

# ax[2].imshow(_test_cube_1[0]-_binned_cube[0], interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[2].set_title('Difference', fontsize = '16')
# ax[2].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)




Seeing how to intepertate missing nan value in this page:https://modelhelptokyo.wordpress.com/2017/10/25/how-to-interpolate-missing-values-2d-python/

In [ ]:
# ## TODO - convert this into a function
def __interploate_arr (array, method = 'cubic'): 
    
    # Settting up the grid for intepoloation
    x = np.arange(0, array.shape[1])
    y = np.arange(0, array.shape[0])
    
    xx, yy = np.meshgrid(x, y)
    
    # Masking invalid values
    array = np.ma.masked_invalid(array)
    
    # Getting only the valid values
    x1 = xx[~array.mask]
    y1 = yy[~array.mask]
    newarr = array[~array.mask]

    newarr = array[~array.mask]

    arr = scipy.interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy),
                                 method=method)
    return arr 


def interploate_nan (array, method = 'cubic'):
    """
    Wrapper function for the interploation for 3D image cube of 2D image cube 
    
    --> if arr.dim = 3: 
            call the private function in loop
            although the interploate function can work in 3D but I am not sure about how does it 
            work in 3D. 
            
            If we treat each 2D image independent from time/wavelength 
    """
    
    _method = ['cubic', '']
    
    ndim = len(array.shape)
    
    if ndim == 3: 
        # it is a 3D cube
        for i in range (array.shape[0]): 
            array[i] = __interploate_arr (array[i], method)
            
        return array
    
    elif ndim == 2:
        # it is a 2D image, or a frame, if you would like to put it that way
        array = __interploate_arr(array, method)
        return array
    
    else: 
        raise Exception("Dimension of %d image is not supported" %ndim)

    
# # array = _binned_cube[1]

# # x = np.arange(0, array.shape[1])
# # y = np.arange(0, array.shape[0])
# # #mask invalid values
# # array = np.ma.masked_invalid(array)
# # xx, yy = np.meshgrid(x, y)
# # #get only the valid values
# # x1 = xx[~array.mask]
# # y1 = yy[~array.mask]
# # newarr = array[~array.mask]

# # GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
# #                           (xx, yy),
# #                              method='cubic')


# _out = interploate_nan(_binned_cube)

In [ ]:
# fig, ax = plt.subplots(1,3)
# xlimt = 200//2 * muse_pix_scale
# ylimt = 200//2 * muse_pix_scale

# xlimt = np.shape(cube[0])[0]//2 * muse_pix_scale
# ylimt = np.shape(cube[0])[1]//2 * muse_pix_scale

# # ax[0].imshow(cube[0],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])

# ax[0].imshow(_binned_cube[0], interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[0].set_title('Original Image', fontsize = '16')
# ax[0].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

# ax[1].imshow(_out[0],interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[1].set_title('Filled image', fontsize = '16')
# ax[1].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)


# ax[2].imshow(_binned_cube[1]-_out[1], interpolation='nearest',extent=[-xlimt,xlimt,ylimt,-ylimt])
# ax[2].set_title('Original Image', fontsize = '16')
# ax[2].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)


In [ ]:
wdir = "/Users/alau/Data/MUSE_DATA/HD_146233/"

data_cube = ["HD_146233_cube_1_binned_10",\
        "HD_146233_cube_2_binned_10"]



name = data_cube[1]

_cube = fits.open(wdir+name+".fits")
_cube.info()

cube = _cube[1].data
_cube.pop(2)
_cube.info()

In [ ]:
stat = _cube[2].data

plt.imshow(stat[100])

In [ ]:
_out = interploate_nan(cube)

In [ ]:
plt.imshow(_out[0])

In [ ]:
_cube[1].data = _out

In [ ]:
plt.imshow(_cube[1].data[0])

In [ ]:
name = "HD_146233_clean_cube_2"
array.save_fits(_cube,name,wdir)